# Twitter US Airline Sentiment

## ML Pipeline Preparation

### Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with read_sql_table
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
import pickle
import sys

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/student/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/student/nltk_data...


True

In [2]:
# load data from database
engine = create_engine('sqlite:///AirlineSentiment.db')
df = pd.read_sql_table('tweets', engine)
X = df['text']
Y = df['airline_sentiment']

### Tokenization Process
- Normalize text and remove punctuation
- Tokenize text
- Remove stop words
- Lemmatize verbs
- Return the list of clean tokens

In [3]:
def tokenize(text):
    # Normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words("english")]

    # Lemmatize verbs by specifying pos
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### Build machine learning pipeline for Random Forest Classifier
- Convert text to word count vectors
- Convert word count vectors to TF-IDF representation
- Instantiate the RandomForest Classifier

In [4]:
rf_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to word count vectors
    ('tfidf', TfidfTransformer()),  # Convert word count vectors to TF-IDF representation
    ('clf', RandomForestClassifier())  # Classifier using RandomForest
])

### Train Model
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train pipeline
rf_pipeline.fit(X_train, Y_train)

/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### Test model
Report the accuracy, f1 score, precision and recall

In [6]:
# Make predictions
Y_pred = rf_pipeline.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

              precision    recall  f1-score   support

    negative       0.80      0.92      0.86      1880
     neutral       0.64      0.46      0.53       582
    positive       0.74      0.57      0.64       459

    accuracy                           0.77      2921
   macro avg       0.73      0.65      0.68      2921
weighted avg       0.76      0.77      0.76      2921



### Improve model
Use grid search to find better parameters.

In [7]:
# Define parameters for grid search
rf_parameters = {
    'clf__n_estimators': [50, 100],
    'clf__min_samples_split': [2, 5]
}

# Perform grid search
rf_cv = GridSearchCV(rf_pipeline, param_grid=rf_parameters, verbose=3)
rf_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END clf__min_samples_split=2, clf__n_estimators=50;, score=0.756 total time=  28.3s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END clf__min_samples_split=2, clf__n_estimators=50;, score=0.745 total time=  28.3s


...

[CV 4/5] END clf__min_samples_split=5, clf__n_estimators=100;, score=0.769 total time=  29.9s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END clf__min_samples_split=5, clf__n_estimators=100;, score=0.740 total time=  30.2s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__min_samples_split': [2, 5],
                         'clf__n_estimators': [50, 100]},
             verbose=3)

### Test model
Show the accuracy, precision, and recall of the tuned model.

In [8]:
# Predict on test data
Y_pred = rf_cv.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

# Calculate overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print("Overall Accuracy:", accuracy)

              precision    recall  f1-score   support

    negative       0.80      0.94      0.86      1880
     neutral       0.66      0.42      0.51       582
    positive       0.74      0.56      0.64       459

    accuracy                           0.77      2921
   macro avg       0.73      0.64      0.67      2921
weighted avg       0.76      0.77      0.76      2921

Overall Accuracy: 0.7737076343717905


### Build machine learning pipeline for Gradient Boosting Classifier

In [9]:
gb_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to word count vectors
    ('tfidf', TfidfTransformer()),  # Convert word count vectors to TF-IDF representation
    ('clf', GradientBoostingClassifier())  # Classifier using GradientBoosting
])

### Train model

In [10]:
# Train pipeline
gb_pipeline.fit(X_train, Y_train)

/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier())])

### Test model

In [11]:
# Make predictions
Y_pred = gb_pipeline.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

              precision    recall  f1-score   support

    negative       0.74      0.96      0.84      1880
     neutral       0.75      0.15      0.25       582
    positive       0.74      0.57      0.64       459

    accuracy                           0.74      2921
   macro avg       0.74      0.56      0.58      2921
weighted avg       0.74      0.74      0.69      2921



### Improve Gradient Boosting Model
Use grid search to find better parameters.

In [12]:
gb_parameters = {
    'clf__n_estimators': [50, 100],
    'clf__min_samples_split': [2, 5],
}

# Perform grid search
gb_cv = GridSearchCV(gb_pipeline, param_grid=gb_parameters, verbose=3)
gb_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END clf__min_samples_split=2, clf__n_estimators=50;, score=0.711 total time=  28.1s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END clf__min_samples_split=2, clf__n_estimators=50;, score=0.705 total time=  27.8s


...

[CV 4/5] END clf__min_samples_split=5, clf__n_estimators=100;, score=0.709 total time=  32.8s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END clf__min_samples_split=5, clf__n_estimators=100;, score=0.703 total time=  33.0s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', GradientBoostingClassifier())]),
             param_grid={'clf__min_samples_split': [2, 5],
                         'clf__n_estimators': [50, 100]},
             verbose=3)

### Test model
Show the accuracy, precision, and recall of the tuned model.

In [13]:
# Predict on test data
Y_pred = gb_cv.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

# Calculate overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print("Overall Accuracy:", accuracy)

              precision    recall  f1-score   support

    negative       0.74      0.97      0.84      1880
     neutral       0.77      0.15      0.25       582
    positive       0.73      0.58      0.65       459

    accuracy                           0.74      2921
   macro avg       0.75      0.56      0.58      2921
weighted avg       0.75      0.74      0.69      2921

Overall Accuracy: 0.7422115713796645


### Build machine learning pipeline for Naive Bayes Classifier with Grid Search

In [14]:
# Define pipeline
nb_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to word count vectors
    ('tfidf', TfidfTransformer()),  # Convert word count vectors to TF-IDF representation
    ('clf', MultinomialNB())  # Classifier using Naive Bayes
])

### Train model using grid search

In [15]:
nb_parameters = {
    'clf__alpha': [0.1, 0.5, 1.0],  # Smoothing parameter (alpha)
}

# Perform grid search
nb_cv = GridSearchCV(nb_pipeline, param_grid=nb_parameters, verbose=3)
nb_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END ....................clf__alpha=0.1;, score=0.751 total time=  24.0s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END ....................clf__alpha=0.1;, score=0.739 total time=  25.3s


...

[CV 4/5] END ....................clf__alpha=1.0;, score=0.677 total time=  25.5s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END ....................clf__alpha=1.0;, score=0.676 total time=  24.7s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.1, 0.5, 1.0]}, verbose=3)

### Test Model

In [16]:
# Predict on test data
Y_pred = nb_cv.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

# Calculate overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print("Overall Accuracy:", accuracy)

              precision    recall  f1-score   support

    negative       0.77      0.96      0.85      1880
     neutral       0.65      0.34      0.44       582
    positive       0.79      0.46      0.58       459

    accuracy                           0.76      2921
   macro avg       0.74      0.58      0.62      2921
weighted avg       0.75      0.76      0.73      2921

Overall Accuracy: 0.7569325573433756


- It's clear that the random forest model performed better than models built with Gradient Boosting and Naive Bayes. Therefore the random model will be saved for deployment.

### Build machine learning pipeline for Decision Tree Classifier with Grid Search

In [17]:
# Define pipeline
dt_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to word count vectors
    ('tfidf', TfidfTransformer()),  # Convert word count vectors to TF-IDF representation
    ('clf', DecisionTreeClassifier())  # Classifier using Decision Tree
])

### Train model using grid search

In [18]:
# Define parameters for grid search
dt_parameters = {
    'clf__min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
    'clf__min_samples_leaf': [1, 2]  # Minimum number of samples required to be at a leaf node
}

# Perform grid search
dt_cv = GridSearchCV(dt_pipeline, param_grid=dt_parameters, verbose=3)
dt_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END clf__min_samples_leaf=1, clf__min_samples_split=2;, score=0.709 total time=  26.4s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END clf__min_samples_leaf=1, clf__min_samples_split=2;, score=0.695 total time=  26.3s


...

[CV 4/5] END clf__min_samples_leaf=2, clf__min_samples_split=5;, score=0.692 total time=  24.9s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END clf__min_samples_leaf=2, clf__min_samples_split=5;, score=0.682 total time=  24.8s


/opt/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f074a1df880>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', DecisionTreeClassifier())]),
             param_grid={'clf__min_samples_leaf': [1, 2],
                         'clf__min_samples_split': [2, 5]},
             verbose=3)

### Test Model

In [19]:
# Predict on test data
Y_pred = dt_cv.predict(X_test)

# print classification report
print(classification_report(Y_test, Y_pred, target_names=['negative', 'neutral', 'positive']))

# Calculate overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print("Overall Accuracy:", accuracy)

              precision    recall  f1-score   support

    negative       0.82      0.81      0.81      1880
     neutral       0.48      0.49      0.48       582
    positive       0.59      0.61      0.60       459

    accuracy                           0.71      2921
   macro avg       0.63      0.63      0.63      2921
weighted avg       0.71      0.71      0.71      2921

Overall Accuracy: 0.7110578568983225


### Save the best model
Since the Random Forest model performed best, it will be saved and used for deployment.

In [20]:
# Save the random forest model to a file
with open('best_model.pkl', 'wb') as f:
    pickle.dump(rf_pipeline, f)